In [8]:
import pandas as pd
import re

In [ ]:
# Group 1: Datasets that have only 3 colunms (Subject, Body, Label)
try:
    df_enron = pd.read_csv('../dataset/Enron.csv', encoding='latin-1', usecols=['subject', 'body', 'label'])
    df_ling = pd.read_csv('../dataset/Ling.csv', encoding='latin-1', usecols=['subject', 'body', 'label'])
except FileNotFoundError as e:
    print(f"Error loading file for Merge 1: {e}")

# Group 2: Rich Metadata Datasets that have 7 colunms (Sender, Date, Subject, Body, Label, etc.)
try:
    df_ceas = pd.read_csv('../dataset/CEAS_08.csv', encoding='latin-1')
    df_nazario = pd.read_csv('../dataset/Nazario.csv', encoding='latin-1')
    df_nifr = pd.read_csv('../dataset/Nigerian_Fraud.csv', encoding='latin-1')
    df_spas = pd.read_csv('../dataset/SpamAssasin.csv', encoding='latin-1')
except FileNotFoundError as e:
    print(f"Error loading file for Merge 2: {e}")

# 1. Merge 1: Combine Enron and Ling (mdf_1)
mdf_1 = pd.concat([df_enron, df_ling], ignore_index=True)
print(f"Merged mdf_1 size: {len(mdf_1)}")

# 2. Merge 2: Combine CEAS, Nazario, NifR, and SpAs (mdf_2)
mdf_2 = pd.concat([df_ceas, df_nazario, df_nifr, df_spas], ignore_index=True)
print(f"Merged mdf_2 size: {len(mdf_2)}")

# 3. Final Integration: Combine mdf_1 and mdf_2 into the final dataset
# Note: This will create many NaN (missing) values for columns mdf_1 doesn't have (like sender, date).

# I solved that later in the code
df_full = pd.concat([mdf_1, mdf_2], ignore_index=True)

print(f"\n--- Final Full Dataset Created ---")
print(f"Total records in df_full: {len(df_full)}")
print("Columns in the final dataset (Note NaNs will be present):")
print(df_full.columns.tolist())

# Now df_full is ready for the feature engineering steps (Date, URL, Sender Entropy).

Merged mdf_1 size: 32626
Merged mdf_2 size: 49860

--- Final Full Dataset Created ---
Total records in df_full: 82486
Columns in the final dataset (Note NaNs will be present):
['subject', 'body', 'label', 'sender', 'receiver', 'date', 'urls']


In [3]:
df_full.head(5)

,subject,body,label,sender,receiver,date,urls
0,"hpl nom for may 25 , 2001",( see attached file : hplno 525 . xls )\r\n- h...,0,NaN,NaN,NaN,NaN
1,re : nom / actual vols for 24 th,- - - - - - - - - - - - - - - - - - - - - - fo...,0,NaN,NaN,NaN,NaN
2,"enron actuals for march 30 - april 1 , 201","estimated actuals\r\nmarch 30 , 2001\r\nno flo...",0,NaN,NaN,NaN,NaN
3,"hpl nom for may 30 , 2001",( see attached file : hplno 530 . xls )\r\n- h...,0,NaN,NaN,NaN,NaN
4,"hpl nom for june 1 , 2001",( see attached file : hplno 601 . xls )\r\n- h...,0,NaN,NaN,NaN,NaN


In [ ]:
# Save df_full to a CSV file for future use
df_full.to_csv('../dataset/dataset.csv', index=False, encoding='latin-1')

In [9]:
# Define the IP Regex just for checking the extracted list elements
ip_regex = r'^\s*http[s]?://\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}'

# Define a general URL regex to get all URLs in a list and get also their count
url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

# Extract all URLs in each email into a column called 'urls_in_email'
df_full['urls_in_email'] = df_full['body'].apply(lambda x: re.findall(url_regex, str(x)))

#URL Cont: take the length of urls in urls_in_email column
df_full['url_count_derived'] = df_full['urls_in_email'].apply(len).fillna(0).astype(int)

def check_for_ip(url_list):
    """Returns 1 if any URL in the list starts with an IP address, 0 otherwise."""
    if not url_list:
        return 0

    for url in url_list:
        if re.match(ip_regex, url):
            return 1
    return 0

df_full['has_ip_url_derived'] = df_full['urls_in_email'].apply(check_for_ip)

print(" Final URL Numerical Features Derived.")

print(df_full[['label', 'url_count_derived', 'has_ip_url_derived', 'urls_in_email']].head(10).to_markdown(index=False))

 Final URL Numerical Features Derived.
|   label |   url_count_derived |   has_ip_url_derived | urls_in_email   |
|--------:|--------------------:|---------------------:|:----------------|
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |
|       0 |                   0 |                    0 | []              |


In [10]:
# Access the list of URLs for the email at index 82481 why that row ?
#I just noticed it has alot of urls
detailed_email = df_full.iloc[82481]
print(f"Detailed email at index 82481:\nLabel: {detailed_email['label']}\nNum of URLs: {detailed_email['url_count_derived']}\nHas IP URL: {detailed_email['has_ip_url_derived']}")
urls_for_email_82481 = df_full.iloc[82481]['urls_in_email']
print(f"URLs found in email 82481: {urls_for_email_82481}")

Detailed email at index 82481:
Label: 1
Num of URLs: 14
Has IP URL: 0
URLs found in email 82481: ['http://images.pcdi-homestudy.com/ads/e277/unisex_header.gif)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://images.pcdi-homestudy.com/ads/e277/unisex_5up.jpg)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://images.pcdi-homestudy.com/ads/e277/1pixel.gif)', 'http://www.frugaljoe.com/logo.jpg)', 'http://www.frugaljoe.com/dot.gif)', 'http://www.frugaljoe.com/unsubscribe.php?eid=396068\\', 'http://www.330w.com/open/open.php?eid=jm@netnoteinc.com&oid=Professional_Career_Development_Institute-1-12032002-HTML&custid=frugaljoe)']


In [11]:
# check if there is row has ip_url_derived with value>0
df_full[df_full['has_ip_url_derived']>0]

,subject,body,label,sender,receiver,date,urls,urls_in_email,url_count_derived,has_ip_url_derived
32777,Your order,"Britney spears p0rn video, Jennifer Lopez pict...",1,Buddy Oneal <Buddy@gogimagog.com>,user5@gvc.ceas-challenge.cc,"Wed, 06 Aug 2008 02:40:05 +0400",1.0,[http://67.159.26.177/moves/rel.php],1,1
33507,[soaplite] Digest Number 1746,SOAP::Lite for Perl (soaplite.com) SOAP:...,0,vtffajla@yahoogroups.com,vtffajla@yahoogroups.com,"Wed, 06 Aug 2008 00:16:03 +0000",1.0,"[http://localhost/hibye.cgi'), http://127.0.1....",2,1
39621,Virtualization is here and now,\r\n==========================================...,0,BEA Systems <ckns@go.bea.com>,user6@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 21:32:49 -0500",1.0,[http://www.bea.com/framework.jsp?CNT=pr01898....,36,1
39650,[UAI] Special Session - Advances in Processing...,**********************************************...,0,Peter Tino <v.bngq@cs.bham.ac.uk>,faq@cs.orst.edu,"Wed, 06 Aug 2008 02:32:47 +0000",1.0,"[http://202.197.224.16/nc2005/, http://www.xtu...",4,1
40537,Re: [python-win32] Windows Vista slow down Pyt...,Hi! \n\n\nIf you replace localhost by 127.0...,0,Michel Claveau <gt@mclaveau.com>,kpitck-aew45@python.org,"Wed, 06 Aug 2008 05:28:33 +0100",1.0,"[http://127.0.0.1:8080, http://mail.python.org...",2,1
...,...,...,...,...,...,...,...,...,...,...
82348,Would you like a $250 check?,![](http://track.optinllc.com/tev.asp?eid=2017...,1,opportunities@optinllc.com,yyyy@netnoteinc.com,"Sat, 3 Aug 2002 02:13:07",1.0,"[http://track.optinllc.com/tev.asp?eid=20175),...",3,1
82400,>> Best rates on mortgage in the country! <,"![](3D""http://61.129.68.17/mortgage/h1.gif"") ...",1,"""Deborah Shaw"" <687ifsuy@bol.com.br>",<Undisclosed.Recipients@dogma.slashnull.org>,"Mon, 05 Aug 2002 21:19:48 -1700",1.0,"[http://61.129.68.17/mortgage/h1.gif, http://6...",6,1
82409,[ILUG] Re: whats up -colonize,YOU HAVE NEVER SEEN A DILDO SITE LIKE THIS ONE...,1,napkin <napkin@mailhost.com>,"ilug@linux.ie, ilug-request@linux.ie","Tue, 06 Aug 2002 06:50:21 PM -0400",1.0,[http://www.supremewebhosting-online.com/users...,4,1
82455,The best possible mortgage,Opportunity is knocking. Why? Because mortgage...,1,alvalinen33@netscape.net,baaronr@hotmail.com,"Fri, 09 Aug 2002 02:20:15 -0400",1.0,"[http://click.lycos.com/director.asp?id=1, htt...",4,1
